In [7]:
import os
import numpy as np
import scipy.misc
from stylize import stylize
import math
from argparse import ArgumentParser

import vgg
import tensorflow as tf
import numpy as np
from sys import stderr

In [8]:
# default arguments
VGG_PATH = 'imagenet-vgg-verydeep-19.mat'
CONTENT_LAYER = 'relu4_2'

In [17]:
def imread(path):
    return scipy.misc.imread(path).astype(np.float)

def imresize(x, shape):
    return scipy.misc.imresize(x, shape)

In [18]:
class Args(dict):
    pass

options = Args()
options.content = 'examples/selfie_ljy.jpg'
options.library = 'library/vertical/'
options.network = VGG_PATH
options.width = 512

In [27]:
content = imread(options.content)
network = options.network
style_paths = [os.path.join(options.library, f) for f in os.listdir(options.library) if f.endswith('.jpg') or f.endswith('.png')]
width = options.width

target_shape = content.shape

if width is not None:
    #compute new height according to the given width
    target_shape = (int(math.floor(float(content.shape[0]) /
            content.shape[1] * width)), width, content.shape[2])
    content = imresize(content, target_shape)

In [28]:


target_shape

(384, 512, 3)

In [29]:
shape = (1,) + content.shape
content_features = {}

g = tf.Graph()
# compute content features in feedforward mode
with g.as_default(), g.device('/cpu:0'), tf.Session() as sess:
    image = tf.placeholder('float', shape=shape)
    net, mean_pixel = vgg.net(network, image)
    content_pre = np.array([vgg.preprocess(content, mean_pixel)])
    content_features[CONTENT_LAYER] = net[CONTENT_LAYER].eval(
            feed_dict={image: content_pre})

In [30]:
style_features = [{} for _ in style_paths]

# compute style features in feedforward mode
for i in range(len(style_paths)):
    style = imread(style_paths[i])
    style = scipy.misc.imresize(style, target_shape)
    style_shape = (1,) + style.shape
    g = tf.Graph()
    with g.as_default(), g.device('/cpu:0'), tf.Session() as sess:
        image = tf.placeholder('float', shape=style_shape)
        net, _ = vgg.net(network, image)
        style_pre = np.array([vgg.preprocess(style, mean_pixel)])
        style_features[i][CONTENT_LAYER] = net[CONTENT_LAYER].eval(feed_dict={image: style_pre})

In [31]:
distance = []
for i in range(len(style_features)):
    g = tf.Graph()
    with g.as_default(), g.device('/cpu:0'), tf.Session() as sess:
        ###distance between input content and style content
        content_loss = tf.nn.l2_loss(
            style_features[i][CONTENT_LAYER] - content_features[CONTENT_LAYER]) / content_features[CONTENT_LAYER].size
        distance.append(content_loss.eval())

In [32]:
val, idx = min((val, idx) for (idx, val) in enumerate(distance))
print(style_paths[idx])
distance

library/vertical/self-portrait-1888.jpg!Large.jpg


[224838.0,
 168629.61,
 224729.17,
 213819.56,
 174336.39,
 215172.06,
 156204.17,
 262403.22,
 200617.88,
 193308.8,
 264402.75,
 325508.09,
 236156.81,
 238325.64,
 175384.88,
 172256.3,
 204412.94,
 255476.73,
 196205.58,
 198285.64,
 208213.62,
 177285.12,
 218898.58,
 293015.66]